In [4]:
import mud.funs as mdf

# Array handling libraries
import numpy as np
import pandas as pd
import xarray as xr

import datetime

# Statistics libraries
from scipy.stats import uniform, norm
from scipy.stats import gaussian_kde as GKDE

# Plotting libraries
import matplotlib.pyplot as plt

In [5]:
# Default formatting for plots
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [6]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

## <center> Maximal Updated Density (MUD) point estimate inverse problems 
   --- 
Carlos del-Castillo-Negrete
    <br>
    Oden Institute at the University of Texas at Austin
    <br>
    cdelcastillo21@gmail.com 
    
   ---
</center>
    
Based off work from Troy Butler and Michael Pilosov

## Exponential Decay - Single Parameter Estimation Problem 
Consider an exponential decay system with uncertain paramater $\lambda$, which we model with the following differential equation.


<center>
\begin{equation}
\Large
\begin{cases} \frac{\partial u}{\partial t} = \lambda u(t), 0 \lt t \leq 3, \\
                  u(0) = 0.75
\end{cases}
\end{equation}
</center>

The true solution is given by the following equation. 
    
<center>
\begin{equation}
\Large u(t; \lambda) = u_0 \text{exp}(-\lambda t), u_0 = 0.75 
\end{equation}
<\center>

### Goal and assumptions
Our goal is to use any initial knowledge on our parameter space, the assumed exponential decay model above, and data collected from experiments on the real world system to infer the true value $\lambda$, which we set for this example problem to $\lambda^{\dagger}=0.5$. We seek a point estimate of the this true value.

The main assumption we make is that our model is indeed an accurate representation of the physical system we are measuring, and the uncertainties in our experiments are aleatoric, i.e. reducible. Thus we have a noisy measurement device $M(t;\lambda^{\dagger})$ to record the true signal $u(t;\lambda^{\dagger})$ at $N$ points in time, with errors being idenpendent identically distributed Gaussain errors. The data collected can then be described by:

<center>
    \begin{equation}
        \label{eq:d_i}
        \Large d(t_i) = M(t_i, \lambda^{\dagger}) + \xi_i, \xi_i \sim {\mathcal N}(0, \sigma^2), 1 \leq i \leq N
    \end{equation}
</center>

Our goal is to use any initial knowledge on our parameter space, the assumed exponential decay model above, and data collected from our measurement device to infer the true value $\lambda^{\dagger}$. We will do this by building a Quantity of Interest map $Q$ from our observed data and perform data-consistent inversion to compute the updated probability density on our parameter space, given by the following equation

<center>
    \begin{equation}
        \Large
        \pi_{up}(\lambda) = \pi_{in}(\lambda)\frac{\pi_{ob}(Q(\lambda))}{\pi_{pr}(Q(\lambda))}
    \end{equation}
</center>

Here $\pi_{pr}(Q(\lambda))$ corresponds to the push-forward of the initial distribution on our parameter space through the QoI map, while $\pi_{ob}(Q(\lambda))$ is the observed distribution of the QoI map for our system. From the $\pi_{up}(\lambda)$ we can then compute the <bf>Maximal Updated Density</bf> (MUD) point as

<center>
    \begin{equation}
        \Large
        \lambda^{MUD} := \text{argmax}\pi_{up}(\lambda)
    \end{equation}
</center>

Finall for our model problem, we assume our parameter range to be $\Lambda = [0,1]$ and a uniform initial distribution, $\pi_{in}(\lambda)={\mathcal U}([0,1])$.

In [7]:
def exp_decay_one(
    u_0=0.75,
    time_range=[0, 3.0],
    domain=[0, 1],
    num_samples=10000,
    lambda_true=0.5,
    N=20,
    t_start=1.0,
    sampling_freq=100.0,
):

    time = np.linspace(time_range[0], time_range[1], 1000)
    times = np.arange(t_start, time_range[1], 1 / sampling_freq)[0:N]

    u_t_lambda = lambda t, l: u_0 * np.exp(-np.outer(l, t))

    mn = np.min(domain, axis=1)
    mx = np.max(domain, axis=1)
    initial = uniform(loc=mn, scale=mx - mn)
    lambda_samples = initial.rvs(size=num_samples)

    true = u_t_lambda(times, lambda_true)[0]
    predicted = u_t_lambda(times, lambda_samples)

    return times, lambda_samples, predicted, true

In [8]:
# Scenario 1 -> Take 30 measurements at 10 Hz starting at t=0
u_0 = 0.75  # Initial condition
time_range = [0, 10.0]  # Time range (secs)
domain = [[0, 1]]  # Domain of possible lambda values
lambda_true = 0.5  # True value of lambda
num_samples = 100  # Number of parameter samples.
N = 30  # Number of measurements from sample/true trajectories to take.
t_start = 0.0  # Time at which to start taking measurements (secs)
sampling_freq = 100.0  # Sampling frequency (Hz)
sd = np.sqrt(1.507e-3)  # Std Deviation for adding noise to measurements

In [9]:
ts, lam, qoi, qoi_true = exp_decay_one(
    u_0=u_0,
    time_range=time_range,
    domain=domain,
    lambda_true=lambda_true,
    num_samples=num_samples,
    N=N,
    t_start=t_start,
    sampling_freq=sampling_freq,
)
ts.shape, lam.shape, qoi.shape, qoi_true.shape

((30,), (100,), (100, 30), (30,))

In [10]:
# Build and solve mud problem
exp_decay_mud = mdf.mud_problem(
    domain=domain, lam=lam, qoi=qoi, sd=sd, qoi_true=qoi_true, num_obs=N
)
exp_decay_mud.estimate()

array([0.51226471])

In [11]:
# Scenario 1 -> Split previous data into 3 iterations of 10 measurements at 10 Hz starting at t=0
# Use weights from previous iteration to weigh next iteration.
num_splits = 3
ts_it = np.split(ts, num_splits)
qoi_it = np.split(qoi, num_splits, axis=1)
qoi_true_it = np.split(qoi_true, num_splits)

weights = None
mud_solves = []
for i in range(num_splits):
    mud_it = mdf.mud_problem(
        domain=domain,
        lam=lam,
        qoi=qoi_it[i],
        sd=sd,
        qoi_true=qoi_true_it[i],
        num_obs=int(N / num_splits),
        weights=weights,
    )
    res = mud_it.estimate()

    print(f"Lambda estimate = {res}")
    weights = mud_it._r
    mud_solves.append(mud_it)

Lambda estimate = [0.9700813]
Lambda estimate = [0.60955897]
Lambda estimate = [0.3443892]
